# 국가별 알콜 섭취량 데이터 분석
- 데이터 읽기
- 데이터 정보 확인
- 컬럼명 한글로 변경
    - 원본 데이터 컬럼 정보
    - country : 국가명
    - beer_servings : 맥주 소비량
    - spirit_servings : 증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)
    - wine_servings : 와인 소비량
    - total_litres_of_pure_alcohol : 순수 알코올
    - continent : 대륙
- 결측치 확인
    - `대륙`에 대한 정보가 부족한 것을 확인할 수 있다.
    - drinks.csv 파일을 열어 원인을 확인한다.
    - 대륙의 종류
        - 아시아, 유럽, 아프리카, 북아메리카, 남아메리카, 오세아니아
        - AS, EU, AF, NA, SA, OC
- 결측치 해결
    - 'NA'를 NaN(결측치)로 읽는 것을 방지
    - pd.read_csv 함수의 na_filter=False 파라메터 : 결측치를 읽어올지 여부 지정
    - 옵션 확인 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 
- 통계값 구하기
    - 전세계 맥주, 증류주, 와인, 알코올의 소비에 대한 '평균', '최대값', '최소값' 구하기
- 주류 소비량 비교
    - 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬
    - 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색
    - 전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색
    - 맥주를 가장 많이 소비하는 국가 확인
    - 증류주를 가장 많이 소비하는 국가 확인
    - 평균 맥주 소비량이 가장 많은 대륙 확인
- 데이터 프레임 조작
    - 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가
    - 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가
        - 알코올비율 = '알코올' / '주류소비량'
        - 알코올비율'이 NaN인 것에 대해 0으로 채우기
    - 알코올비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회
    - DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장
    - 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기
        <pre>
               맥주	    증류주	     와인	      알코올	  주류소비량	알코올비율
     국가						
South Korea	140.000000	16.000000	9.000000	9.800000	165.000000	0.059394
World Wide	106.160622	80.994819	49.450777	4.717098	236.606218	0.026115
</pre>
    - pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구한다.

In [193]:
import pandas as pd
import numpy as np

## 데이터 읽기
csv 파일 가져오기 
- csv 파일은 컬럼 구분이 콤마(,) 로 이루어진 파일
- df = pd.read_csv(파일이름) : csv 형식의 파일을 읽어 DataFrame 객체로 가져옴
- DataFrame.to_csv(파일이름, index=True) : DataFrame을 csv 파일로 저장, index는 제외하고 저장하기 위해서는 index=False로 지정

In [194]:
df = pd.read_csv('./drinks.csv')

df.head()

OSError: [Errno 22] Invalid argument: './drinks.csv'

In [4]:
df.shape

(193, 6)

## 컬럼명 한글로 변경

컬럼명 한글로 변경
원본 데이터 컬럼 정보
country : 국가명
beer_servings : 맥주 소비량
spirit_servings : 증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)
wine_servings : 와인 소비량
total_litres_of_pure_alcohol : 순수 알코올
continent : 대륙

In [6]:
df.columns = ['국가명','맥주 소비량','증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)','와인 소비량','순수 알코올','대륙']

In [7]:
df.columns

Index(['국가명', '맥주 소비량', '증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)',
       '와인 소비량', '순수 알코올', '대륙'],
      dtype='object')

## 결측치 확인

In [8]:
df.isna()

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
188,False,False,False,False,False,False
189,False,False,False,False,False,False
190,False,False,False,False,False,False
191,False,False,False,False,False,False


In [11]:
np.sum(df.isna())

국가명                                             0
맥주 소비량                                          0
증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)     0
와인 소비량                                          0
순수 알코올                                          0
대륙                                             23
dtype: int64

In [24]:
#data[data['대여소ID'].isna()]
df['대륙'].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
188    False
189    False
190    False
191    False
192    False
Name: 대륙, Length: 193, dtype: bool

In [25]:
df[df['대륙'].isna()]

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙
5,Antigua & Barbuda,102,128,45,4.9,NaN
11,Bahamas,122,176,51,6.3,NaN
14,Barbados,143,173,36,6.3,NaN
17,Belize,263,114,8,6.8,NaN
32,Canada,240,122,100,8.2,NaN
41,Costa Rica,149,87,11,4.4,NaN
43,Cuba,93,137,5,4.2,NaN
50,Dominica,52,286,26,6.6,NaN
51,Dominican Republic,193,147,9,6.2,NaN
54,El Salvador,52,69,2,2.2,NaN


## 결측치 해결

- 'NA'를 NaN(결측치)로 읽는 것을 방지
    - pd.read_csv(파일이름, na_filter=False) : 결측치를 읽어올지 여부 지정
    - 옵션 확인 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 


- 결측치를 다른 값으로 채우기 함
    - DataFrame.fillna(값) 또는 Series.fillna(값) : 결측값을 특정 값으로 채우기 
    - 연속형 데이터 : 0으로 채우기, 평균값으로 채우기, 범주별 평균값 채우기 등
    - 범주형 데이터 : 다른 범주로 만들어 채우기 

In [27]:
df = df.fillna('NA')

In [29]:
np.sum(df.isna())

국가명                                            0
맥주 소비량                                         0
증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)    0
와인 소비량                                         0
순수 알코올                                         0
대륙                                             0
dtype: int64

## 통계값 구하기
- DataFrame.describe() : 컬럼별 count, mean, std, min, max, Q1, Q2, Q3 등의 통계값 구하기

전세계 맥주, 증류주, 와인, 알코올의 소비에 대한 '평균', '최대값', '최소값' 구하기


In [30]:
df.describe()

,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올
count,193.000000,193.000000,193.000000,193.000000
mean,106.160622,80.994819,49.450777,4.717098
std,101.143103,88.284312,79.697598,3.773298
min,0.000000,0.000000,0.000000,0.000000
25%,20.000000,4.000000,1.000000,1.300000
50%,76.000000,56.000000,8.000000,4.200000
75%,188.000000,128.000000,59.000000,7.200000
max,376.000000,438.000000,370.000000,14.400000


## 주류 소비량 비교



와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬
맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색
전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색
맥주를 가장 많이 소비하는 국가 확인
증류주를 가장 많이 소비하는 국가 확인
평균 맥주 소비량이 가장 많은 대륙 확인

In [31]:
df.columns

Index(['국가명', '맥주 소비량', '증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)',
       '와인 소비량', '순수 알코올', '대륙'],
      dtype='object')

In [32]:
df.head()

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬

In [36]:
df.groupby('국가명').filter(lambda x: x['와인 소비량'] > x['맥주 소비량']+x['증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)']).sort_values('대륙')

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙
55,Equatorial Guinea,92,0,233,5.8,AF
148,Sao Tome & Principe,56,38,140,4.2,AF
92,Laos,62,0,123,6.2,AS
171,Timor-Leste,1,1,4,0.1,AS
61,France,127,151,370,11.8,EU
83,Italy,85,42,237,6.5,EU
136,Portugal,194,67,339,11.0,EU
6,Argentina,193,25,221,8.3,SA
185,Uruguay,115,35,220,6.6,SA


In [ ]:
맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색

In [50]:
df.groupby('국가명').filter(lambda x: x['맥주 소비량'] > 230 and x['와인 소비량'] >230)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [54]:
df[df['맥주 소비량'] > 230][df['와인 소비량'] > 230]

C:\Users\user\AppData\Local\Temp\ipykernel_13972\4176599407.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['맥주 소비량'] > 230][df['와인 소비량'] > 230]


,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙
3,Andorra,245,138,312,12.4,EU
99,Luxembourg,236,133,271,11.4,EU
156,Slovenia,270,51,276,10.6,EU


전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색

In [ ]:
df[df.groupby('대륙').]

In [94]:
df.groupby('국가명').filter(lambda x: x['순수 알코올'] > df['순수 알코올'].mean() )

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙,주류소비량,주류 소비량
1,Albania,89,132,54,4.9,EU,221,221
3,Andorra,245,138,312,12.4,EU,383,383
4,Angola,217,57,45,5.9,AF,274,274
5,Antigua & Barbuda,102,128,45,4.9,NA,230,230
6,Argentina,193,25,221,8.3,SA,218,218
...,...,...,...,...,...,...,...,...
182,United Kingdom,219,126,195,10.4,EU,345,345
183,Tanzania,36,6,1,5.7,AF,42,42
184,USA,249,158,84,8.7,NA,407,407
185,Uruguay,115,35,220,6.6,SA,150,150


In [ ]:
맥주를 가장 많이 소비하는 국가 확인
증류주를 가장 많이 소비하는 국가 확인

In [65]:
df.groupby('국가명')[['맥주 소비량','증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)']].agg(max)

,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)"
국가명,,
Afghanistan,0,0
Albania,89,132
Algeria,25,0
Andorra,245,138
Angola,217,57
...,...,...
Venezuela,333,100
Vietnam,111,2
Yemen,6,0


In [77]:
df.sort_values('맥주 소비량', ascending = False).head(1)

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙
117,Namibia,376,3,1,6.8,AF


In [78]:
df.sort_values('증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)', ascending = False).head(1)

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙
68,Grenada,199,438,28,11.9,NA


평균 맥주 소비량이 가장 많은 대륙 확인

In [81]:
df.groupby('대륙')['맥주 소비량'].mean().sort_values(ascending = False)

대륙
EU    193.777778
SA    175.083333
NA    145.434783
OC     89.687500
AF     61.471698
AS     37.045455
Name: 맥주 소비량, dtype: float64

## 데이터 프레임 조작
- 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가
- 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가
- 알코올비율 = '알코올' / '주류소비량'
- 알코올비율'이 NaN인 것에 대해 0으로 채우기
- 알코올비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회
- DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로 정렬하여 df로 저장
- 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기

### 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가

In [119]:
df['주류 소비량'] = df[df.columns[1:3]].sum(axis=1)

In [120]:
df.columns

Index(['국가명', '맥주 소비량', '증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)',
       '와인 소비량', '순수 알코올', '대륙', '주류소비량', '주류 소비량', '알코올 비율'],
      dtype='object')

In [121]:
df.head()

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙,주류소비량,주류 소비량,알코올 비율
0,Afghanistan,0,0,0,0.0,AS,0,0,0.000000
1,Albania,89,132,54,4.9,EU,221,221,0.022172
2,Algeria,25,0,14,0.7,AF,25,25,0.028000
3,Andorra,245,138,312,12.4,EU,383,383,0.032376
4,Angola,217,57,45,5.9,AF,274,274,0.021533


### 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가

In [122]:
df['알코올 비율']= df['순수 알코올']/df['주류 소비량']

In [123]:
df.head()

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙,주류소비량,주류 소비량,알코올 비율
0,Afghanistan,0,0,0,0.0,AS,0,0,NaN
1,Albania,89,132,54,4.9,EU,221,221,0.022172
2,Algeria,25,0,14,0.7,AF,25,25,0.028000
3,Andorra,245,138,312,12.4,EU,383,383,0.032376
4,Angola,217,57,45,5.9,AF,274,274,0.021533


### 알코올비율'이 NaN인 것에 대해 0으로 채우기

In [124]:
sum(df['알코올 비율'].isna())

13

In [125]:
df['알코올 비율'] = df['알코올 비율'].fillna(0)

In [126]:
df.head()

,국가명,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,대륙,주류소비량,주류 소비량,알코올 비율
0,Afghanistan,0,0,0,0.0,AS,0,0,0.000000
1,Albania,89,132,54,4.9,EU,221,221,0.022172
2,Algeria,25,0,14,0.7,AF,25,25,0.028000
3,Andorra,245,138,312,12.4,EU,383,383,0.032376
4,Angola,217,57,45,5.9,AF,274,274,0.021533


### 알코올비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회

In [134]:
df.sort_values('알코올 비율', ascending = False).head(5)[['국가명','주류 소비량','알코올 비율']]

,국가명,주류 소비량,알코올 비율
63,Gambia,8,0.300000
153,Sierra Leone,28,0.239286
124,Nigeria,47,0.193617
179,Uganda,54,0.153704
142,Rwanda,45,0.151111


### DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로 정렬하여 df로 저장

In [138]:
#df.set_index(열이름, inplace = True)
df = df.set_index(['대륙','국가명']).sort_index()

In [139]:
df.head()

맥주 소비량  증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)  와인 소비량  \
대륙 국가명                                                                         
AF Algeria           25                                            0      14   
   Angola           217                                           57      45   
   Benin             34                                            4      13   
   Botswana         173                                           35      35   
   Burkina Faso      25                                            7       7   

                 순수 알코올  주류소비량  주류 소비량    알코올 비율  
대륙 국가명                                            
AF Algeria          0.7     25      25  0.028000  
   Angola           5.9    274     274  0.021533  
   Benin            1.1     38      38  0.028947  
   Botswana         5.4    208     208  0.025962  
   Burkina Faso     4.3     32      32  0.134375

### 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기

In [147]:
sr1 = df.mean()
sr1

맥주 소비량                                         106.160622
증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)     80.994819
와인 소비량                                          49.450777
순수 알코올                                           4.717098
주류소비량                                          187.155440
주류 소비량                                         187.155440
알코올 비율                                           0.031394
dtype: float64

In [149]:
sr2 = df.loc[('AS','South Korea')]
sr2

맥주 소비량                                         140.000000
증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)     16.000000
와인 소비량                                           9.000000
순수 알코올                                           9.800000
주류소비량                                          156.000000
주류 소비량                                         156.000000
알코올 비율                                           0.062821
Name: (AS, South Korea), dtype: float64

In [152]:
df_compare = pd.DataFrame({'total': sr1,
                          'South Korea': sr2})

df_compare.T

,맥주 소비량,"증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)",와인 소비량,순수 알코올,주류소비량,주류 소비량,알코올 비율
total,106.160622,80.994819,49.450777,4.717098,187.15544,187.15544,0.031394
South Korea,140.000000,16.000000,9.000000,9.800000,156.00000,156.00000,0.062821


## 맥주 소비량

### pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구한다.

In [158]:
df.head()

맥주 소비량  증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)  와인 소비량  \
대륙 국가명                                                                         
AF Algeria           25                                            0      14   
   Angola           217                                           57      45   
   Benin             34                                            4      13   
   Botswana         173                                           35      35   
   Burkina Faso      25                                            7       7   

                 순수 알코올  주류소비량  주류 소비량    알코올 비율  
대륙 국가명                                            
AF Algeria          0.7     25      25  0.028000  
   Angola           5.9    274     274  0.021533  
   Benin            1.1     38      38  0.028947  
   Botswana         5.4    208     208  0.025962  
   Burkina Faso     4.3     32      32  0.134375

In [ ]:
(index = 행방행 컬럼, columns = 열방향 컬럼, values= 집계대상 컬럼, aggfunc= 구할 통계값)

In [161]:
df.head()

맥주 소비량  증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)  와인 소비량  \
대륙 국가명                                                                         
AF Algeria           25                                            0      14   
   Angola           217                                           57      45   
   Benin             34                                            4      13   
   Botswana         173                                           35      35   
   Burkina Faso      25                                            7       7   

                 순수 알코올  주류소비량  주류 소비량    알코올 비율  
대륙 국가명                                            
AF Algeria          0.7     25      25  0.028000  
   Angola           5.9    274     274  0.021533  
   Benin            1.1     38      38  0.028947  
   Botswana         5.4    208     208  0.025962  
   Burkina Faso     4.3     32      32  0.134375

In [163]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
df

In [167]:
df.columns

Index(['맥주 소비량', '증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)', '와인 소비량',
       '순수 알코올', '주류소비량', '주류 소비량', '알코올 비율'],
      dtype='object')

In [172]:
df.index

MultiIndex([('AF',                  'Algeria'),
            ('AF',                   'Angola'),
            ('AF',                    'Benin'),
            ('AF',                 'Botswana'),
            ('AF',             'Burkina Faso'),
            ('AF',                  'Burundi'),
            ('AF',               'Cabo Verde'),
            ('AF',                 'Cameroon'),
            ('AF', 'Central African Republic'),
            ('AF',                     'Chad'),
            ...
            ('SA',                   'Brazil'),
            ('SA',                    'Chile'),
            ('SA',                 'Colombia'),
            ('SA',                  'Ecuador'),
            ('SA',                   'Guyana'),
            ('SA',                 'Paraguay'),
            ('SA',                     'Peru'),
            ('SA',                 'Suriname'),
            ('SA',                  'Uruguay'),
            ('SA',                'Venezuela')],
           names=['대륙',

In [191]:
temp = pd.pivot_table(df,index = '대륙', values =['맥주 소비량','와인 소비량'], aggfunc=['mean', 'median', 'max'] )
temp

mean             median           max       
        맥주 소비량      와인 소비량 맥주 소비량 와인 소비량 맥주 소비량 와인 소비량
대륙                                                    
AF   61.471698   16.264151   32.0    2.0    376    233
AS   37.045455    9.068182   17.5    1.0    247    123
EU  193.777778  142.222222  219.0  128.0    361    370
NA  145.434783   24.521739  143.0   11.0    285    100
OC   89.687500   35.625000   52.5    8.5    306    212
SA  175.083333   62.416667  162.5   12.0    333    221